In [5]:
# here, we combine all .csv files into the single file total_state_data.csv
# all .csv files to be combined should be placed in ./Data directory

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df1 = pd.read_csv('./Data/1990-2020-final.csv')
df2 = pd.read_csv('./Data/monthly-emissions-1996-2024-final.csv')

In [8]:
total_df = df1.merge(df2, on=['state','date'], how='inner')

In [9]:
total_df.to_csv('./Data/total_state_data.csv')